# Install and Import Packages

In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to c:\users\tpeiq\appdata\local\temp\pip-req-build-sx1xscab
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit fb2becacbf3c48249ce960f3ef60a862f8fc6707
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to c:\users\tpeiq\appdata\local\temp\pip-install-1tyz16u2\elegantrl_10b574a0708f46c8816ce2d9b6f166d5
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit b2ec84b19a82e6f18b781743d8bf37ca27134238
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-req-build-sx1xscab'
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git 'C:\Users\tpeiq\AppData\Local\Temp\pip-install-1tyz16u2\elegantrl_10b574a0708f46c8816ce2d9b6f166d5'


In [2]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
import itertools
check_and_make_directories([TRAINED_MODEL_DIR])



In [3]:
from datetime import datetime, timedelta

# Set the end date as today
TRADE_END_DATE = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# Set the trade start date as 3 months before the end date
TRADE_START_DATE = (datetime.today() - timedelta(days=300)).strftime('%Y-%m-%d')

# Set the train end date as the trade start date
TRAIN_END_DATE = TRADE_START_DATE

# Set the train start date as 5 years before the train end date
TRAIN_START_DATE = (datetime.strptime(TRAIN_END_DATE, '%Y-%m-%d') - timedelta(days=5*365)).strftime('%Y-%m-%d')

print("TRAIN_START_DATE:", TRAIN_START_DATE)
print("TRAIN_END_DATE:", TRAIN_END_DATE)
print("TRADE_START_DATE:", TRADE_START_DATE)
print("TRADE_END_DATE:", TRADE_END_DATE)

TRAIN_START_DATE: 2019-10-17
TRAIN_END_DATE: 2024-10-15
TRADE_START_DATE: 2024-10-15
TRADE_END_DATE: 2025-08-10


In [4]:
symbols = ['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v']

In [5]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = symbols).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (24820, 8)


In [6]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2019-10-17,56.717457,56.927181,56.293186,56.671654,67585200,aapl,3
1,2019-10-17,31.139999,31.400000,30.860001,31.299999,36884400,amd,3
2,2019-10-17,89.374001,89.942497,89.100998,89.824501,52948000,amzn,3
3,2019-10-17,116.003197,116.697084,114.343109,115.037010,3418300,cat,3
4,2019-10-17,50.439999,53.150002,50.330002,51.759998,2781000,crwd,3


# Preprocess Data

In [7]:
# Step 2: Get the last available date
last_date = df_raw['date'].max()

# Convert last_date to pandas Timestamp if it's a string
if isinstance(last_date, str):
	last_date_dt = pd.to_datetime(last_date)
else:
	last_date_dt = last_date

# Step 3: Filter all rows from the last date (usually one per ticker)
last_day_data = df_raw[df_raw['date'] == last_date].copy()

# Step 4: Duplicate and increment the date by 1 day
next_day = last_date_dt + pd.Timedelta(days=1)
last_day_data['date'] = next_day.strftime('%Y-%m-%d') if isinstance(last_date, str) else next_day

# Step 5: Append the duplicated rows to the original data
df_patched = pd.concat([df_raw, last_day_data], ignore_index=True)

In [8]:
df_patched

Price,date,close,high,low,open,volume,tic,day
0,2019-10-17,56.717457,56.927181,56.293186,56.671654,67585200,aapl,3
1,2019-10-17,31.139999,31.400000,30.860001,31.299999,36884400,amd,3
2,2019-10-17,89.374001,89.942497,89.100998,89.824501,52948000,amzn,3
3,2019-10-17,116.003197,116.697084,114.343109,115.037010,3418300,cat,3
4,2019-10-17,50.439999,53.150002,50.330002,51.759998,2781000,crwd,3
...,...,...,...,...,...,...,...,...
24832,2025-08-09,182.699997,183.300003,180.399994,181.550003,122765700,nvda,4
24833,2025-08-09,67.650002,68.610001,67.419998,68.440002,9667800,pypl,4
24834,2025-08-09,28.080000,28.250000,28.030001,28.070000,23594600,t,4
24835,2025-08-09,329.649994,335.149994,320.980011,321.429993,91006400,tsla,4


In [9]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_patched)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (1460, 8)
Successfully added vix


Successfully added turbulence index


In [10]:
df_raw['date'].max()  # Check the latest date in the raw data

'2025-08-08'

In [11]:
processed['date'].max()  # Check the latest date in the processed data

'2025-08-08'

In [12]:
processed.tic.unique()

array(['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v'],
      dtype=object)

In [13]:

# Align to all dates and tickers
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(), processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))
processed_full = pd.DataFrame(combination, columns=["date", "tic"]) \
    .merge(processed, on=["date", "tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])
processed_full = processed_full.fillna(0)

# Make sure 'date' column is in datetime format
processed_full['date'] = pd.to_datetime(processed_full['date'])

# Sort first for consistency
processed_full = processed_full.sort_values(by=['date', 'tic']).reset_index(drop=True)

# Assign the same index to all rows with the same date
processed_full.index = processed_full.groupby('date').ngroup()

print("Final processed full shape:", processed_full.shape)
print("Final processed full max date:", processed_full['date'].max())

Final processed full shape: (24820, 18)
Final processed full max date: 2025-08-08 00:00:00


In [14]:
processed_full.tic.unique()

array(['aapl', 'amd', 'amzn', 'cat', 'crwd', 'googl', 'gs', 'hd', 'ibm',
       'intc', 'meta', 'msft', 'nvda', 'pypl', 't', 'tsla', 'v'],
      dtype=object)

In [15]:
processed_full['date'].max()

Timestamp('2025-08-08 00:00:00')

# Save the Data

In [16]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

21352
3468


In [17]:
train_path = './data/train_data.csv'
trade_path = './data/trade_data.csv'

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
  train.to_csv(f)

with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
  trade.to_csv(f)

# Load the Data from Part 1

In [18]:
train = pd.read_csv(train_path)

train = train.set_index(train.columns[0])
train.index.names = ['']

## Setup Environment

In [19]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 17, State Space: 171


In [20]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 20,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [21]:
train

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2019-10-17,aapl,56.717457,56.927181,56.293186,56.671654,67585200.0,3.0,0.000000,57.238903,56.468418,100.000000,66.666667,100.000000,56.717457,56.717457,13.790000,0.000000
0,2019-10-17,amd,31.139999,31.400000,30.860001,31.299999,36884400.0,3.0,0.000000,57.238903,56.468418,100.000000,66.666667,100.000000,31.139999,31.139999,13.790000,0.000000
0,2019-10-17,amzn,89.374001,89.942497,89.100998,89.824501,52948000.0,3.0,0.000000,57.238903,56.468418,100.000000,66.666667,100.000000,89.374001,89.374001,13.790000,0.000000
0,2019-10-17,cat,116.003197,116.697084,114.343109,115.037010,3418300.0,3.0,0.000000,57.238903,56.468418,100.000000,66.666667,100.000000,116.003197,116.003197,13.790000,0.000000
0,2019-10-17,crwd,50.439999,53.150002,50.330002,51.759998,2781000.0,3.0,0.000000,57.238903,56.468418,100.000000,66.666667,100.000000,50.439999,50.439999,13.790000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,2024-10-14,nvda,138.038223,139.567870,136.268626,136.438585,232347700.0,0.0,4.831779,138.539687,108.894318,60.340032,182.051114,39.788830,119.492321,117.973432,19.700001,14.408201
1255,2024-10-14,pypl,80.669998,80.980003,79.820000,80.339996,5865400.0,0.0,2.421361,83.078705,73.260296,63.018414,87.037154,34.347094,75.600000,70.654667,19.700001,14.408201
1255,2024-10-14,t,20.567537,20.712651,20.567537,20.664281,33512200.0,0.0,0.228769,21.228500,20.332618,58.568486,4.126073,4.963483,20.628373,19.547288,19.700001,14.408201


### Environment for Training

In [22]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Train Agent

In [23]:
agent = DRLAgent(env = env_train)

# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

## A2C Model

The code above and below provide a healthy framework to add different agent types for training within the same notebook. For this I'll only be training an a2c model, but may add ddpg, ppo, and td3 in later iterations.

In [24]:
model_a2c = agent.get_model("a2c")
model_ppo = agent.get_model('ppo')

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cuda device
Logging to results/a2c
Logging to results/ppo


c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'`

In [25]:
import torch
print(torch.cuda.is_available())
if torch.cuda.is_available():
	print(torch.cuda.get_device_name(0))
else:
	print("CUDA is not available or PyTorch is not compiled with CUDA support.")
	print(torch.__version__)

True
NVIDIA GeForce RTX 3070 Laptop GPU


In [26]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

--------------------------------------
| time/                 |            |
|    fps                | 159        |
|    iterations         | 100        |
|    time_elapsed       | 3          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -24.4      |
|    explained_variance | -0.0273    |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -31.5      |
|    reward             | 0.5452084  |
|    reward_max         | 1.4797827  |
|    reward_mean        | -0.4217824 |
|    reward_min         | -2.3870795 |
|    std                | 1.01       |
|    value_loss         | 9.77       |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 180        |
|    iterations         | 200        |
|    time_elapsed       | 5          |
|    total_timesteps    | 1000       |
| train/                |

In [27]:
trained_a2c.save(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None
trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo") if if_using_ppo else None

In [28]:

if_using_sac = True

model_sac = agent.get_model("sac")


if if_using_sac:
    tmp_path = RESULTS_DIR + '/sac'
    # Only use basic loggers to avoid rollout_buffer errors
    new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
    model_sac.set_logger(new_logger_sac)





{'batch_size': 64, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device
Logging to results/sac


In [29]:
trained_sac = agent.train_model(model=model_sac, tb_log_name='sac', total_timesteps=50000) if if_using_sac else None

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

In [30]:
trained_sac.save(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None

In [31]:
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
import matplotlib.pyplot as plt

In [32]:
train = pd.read_csv('./data/train_data.csv')
trade = pd.read_csv('./data/trade_data.csv')

train = train.set_index(train.columns[0])
train.index.names = ['']
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

In [33]:
if_using_a2c = True
if_using_ddpg = False
if_using_ppo = True
if_using_td3 = False
if_using_sac = True

In [34]:
trained_a2c = A2C.load('trained_models/agent_a2c') if if_using_a2c else None
trained_ddpg = DDPG.load("trained_models/agent_ddpg") if if_using_ddpg else None
trained_ppo = PPO.load("trained_models/agent_ppo") if if_using_ppo else None
trained_td3 = TD3.load("trained_models/agent_td3") if if_using_td3 else None
trained_sac = SAC.load("trained_models/agent_sac") if if_using_sac else None

c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'`

In [35]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 17, State Space: 171


In [36]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 20,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [37]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [38]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c,
    environment = e_trade_gym) if if_using_a2c else (None, None)

hit end!


In [39]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac,
    environment = e_trade_gym) if if_using_sac else (None, None)

hit end!


In [40]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo,
    environment = e_trade_gym) if if_using_ppo else (None, None)

hit end!


In [41]:
print("Env attributes:", dir(e_trade_gym))

Env attributes: ['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_buy_stock', '_get_date', '_initiate_state', '_is_protocol', '_make_plot', '_np_random', '_np_random_seed', '_seed', '_sell_stock', '_update_state', 'action_space', 'actions_memory', 'asset_memory', 'buy_cost_pct', 'close', 'cost', 'data', 'date_memory', 'day', 'df', 'episode', 'get_sb_env', 'get_wrapper_attr', 'has_wrapper_attr', 'hmax', 'initial', 'initial_amount', 'iteration', 'make_plots', 'metadata', 'mode', 'model_name', 'np_random', 'np_random_seed', 'num_stock_shares', 'observation_space', 'previous_state', 'pr

In [42]:
def calculate_sharpe(df_account_value):
    returns = df_account_value['account_value'].pct_change().dropna()
    if returns.std() == 0:
        return 0
    return (252**0.5) * returns.mean() / returns.std()


In [43]:
sharpe_a2c = calculate_sharpe(df_account_value_a2c)
sharpe_sac = calculate_sharpe(df_account_value_sac)
sharpe_ppo = calculate_sharpe(df_account_value_ppo)


In [44]:
def compute_rolling_sharpes(account_value_dfs, window=30):
    import pandas as pd
    import numpy as np

    sharpes = {}
    for name, df in account_value_dfs.items():
        returns = df['account_value'].pct_change().dropna()
        rolling = returns.rolling(window=window)
        rolling_sharpe = (rolling.mean() / rolling.std()) * np.sqrt(252)
        sharpes[name] = rolling_sharpe.fillna(0).values  # shape: [T]
    return sharpes

In [45]:
def run_rolling_best_model(df_actions_dict, account_value_dfs, env, window=30):
    import numpy as np
    import pandas as pd

    # Align by truncating to minimum length
    min_len = min(len(df) for df in df_actions_dict.values())
    df_actions_trimmed = {
        name: df.iloc[:min_len].reset_index(drop=True)
        for name, df in df_actions_dict.items()
    }
    account_value_trimmed = {
        name: df.iloc[:min_len].reset_index(drop=True)
        for name, df in account_value_dfs.items()
    }

    # Rolling sharpes per model
    sharpes = compute_rolling_sharpes(account_value_trimmed, window)

    # Select best model at each timestep
    best_model_at_t = []
    model_names = list(df_actions_trimmed.keys())

    for t in range(min_len):
        best_model = model_names[0]
        best_score = -np.inf
        for name in model_names:
            if t < len(sharpes[name]) and sharpes[name][t] > best_score:
                best_score = sharpes[name][t]
                best_model = name
        best_model_at_t.append(best_model)

    # Build ensemble action list using best model per timestep
    ensemble_rows = []
    for t in range(min_len):
        best = best_model_at_t[t]
        ensemble_rows.append(df_actions_trimmed[best].iloc[t].values)

    # Convert to DataFrame
    df_actions_ensemble = pd.DataFrame(ensemble_rows, columns=df_actions_trimmed[best].columns)

    # Backtest
    obs, _ = env.reset()
    account_values = []

    for i in range(len(df_actions_ensemble)):
        action = df_actions_ensemble.iloc[i].values
        try:
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            account_value = env.asset_memory[-1]
            account_values.append(account_value)
            if done:
                break
        except Exception as e:
            print(f"[ERROR] Step {i} failed: {e}")
            account_values.append(np.nan)
            break

    sim_dates = df_actions_ensemble.index[:len(account_values)]
    df_account_value_ensemble = pd.DataFrame({
        'date': sim_dates,
        'account_value': account_values
    })

    return df_account_value_ensemble, df_actions_ensemble, best_model_at_t


In [46]:
# Input dicts (make sure these are precomputed)
df_actions_dict = {
    'a2c': df_actions_a2c,
    'ppo': df_actions_ppo,
    'sac': df_actions_sac
}

account_value_dfs = {
    'a2c': df_account_value_a2c,
    'ppo': df_account_value_ppo,
    'sac': df_account_value_sac
}

# Run smarter ensemble
df_account_value_smart, df_actions_smart, model_choice = run_rolling_best_model(
    df_actions_dict, account_value_dfs, env=e_trade_gym, window=30
)


In [47]:
df_account_value_smart['date'] = df_account_value_a2c['date']  # Align dates with original A2C

In [48]:
df_account_value_smart

,date,account_value
0,2024-10-15,1.005003e+06
1,2024-10-16,1.004863e+06
2,2024-10-17,1.004388e+06
3,2024-10-18,9.972004e+05
4,2024-10-21,9.959232e+05
...,...,...
198,2025-08-01,1.300420e+06
199,2025-08-04,1.286877e+06
200,2025-08-05,1.301251e+06
201,2025-08-06,1.292619e+06


In [49]:
model_choice

['a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'sac',
 'sac',
 'sac',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'a2c',
 'a2c',
 'a2c',
 'sac',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'ppo',
 'a2c',
 'a2c',
 'a2c',
 'a2c',
 'sac',
 'sac',
 'sac',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'sac',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'ppo',
 'a2c',
 'a2c',
 'a2c',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'ppo',
 'ppo',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',
 'sac',


In [50]:
from collections import Counter

last5 = Counter(model_choice[-5:]).most_common()
full = Counter(model_choice).most_common()

# Simple weighted voting
score_map = {}
for model, count in full:
    score_map[model] = count * 0.4  # weight for full history

for model, count in last5:
    score_map[model] = score_map.get(model, 0) + count * 0.6  # weight for recent

most_common = max(score_map.items(), key=lambda x: x[1])[0]

In [51]:
most_common

'a2c'

In [52]:
confidence = score_map[most_common] / sum(score_map.values())
print(f"Most common model: {most_common} with confidence {confidence:.2f}")

Most common model: a2c with confidence 0.42


In [53]:
import os
import csv

csv_path = 'most_common_model.csv'

# Check if file exists
file_exists = os.path.isfile(csv_path)

# Write or append the most_common value and confidence
with open(csv_path, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    if not file_exists:
        writer.writerow(['most_common', 'confidence'])  # Write header if file does not exist
    writer.writerow([most_common, confidence])

print(f"Appended most_common ('{most_common}') and confidence ({confidence:.2f}) to {csv_path}")

Appended most_common ('a2c') and confidence (0.42) to most_common_model.csv


In [54]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE,TRADE_END_DATE)
print(len(train))

24820


In [55]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 20,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

Stock Dimension: 17, State Space: 171


In [56]:
model_best = agent.get_model(most_common)

tmp_path = RESULTS_DIR + '/best_model'
new_logger_best = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_best.set_logger(new_logger_best)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
Logging to results/best_model


c:\Users\tpeiq\anaconda3\envs\tradingbot310\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [57]:
trained_model_best = agent.train_model(model=model_best, tb_log_name='best', total_timesteps=50000)

---------------------------------------
| time/                 |             |
|    fps                | 191         |
|    iterations         | 100         |
|    time_elapsed       | 2           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -24.2       |
|    explained_variance | -0.415      |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -2.81       |
|    reward             | -1.1888342  |
|    reward_max         | 1.5254114   |
|    reward_mean        | 0.048998117 |
|    reward_min         | -1.1888342  |
|    std                | 1           |
|    value_loss         | 2.49        |
---------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 191        |
|    iterations         | 200        |
|    time_elapsed       | 5          |
|    total_timesteps    | 1000       |
| trai

In [58]:
trained_model_best.save(TRAINED_MODEL_DIR + "/agent_best_model")